In [11]:
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_classic.chains import RetrievalQA
from huggingface_hub import HfFolder
import gradio as gr

In [12]:
from langchain_ollama import OllamaLLM

def llmmodel():
    llm = OllamaLLM(model="phi3:mini", base_url="http://127.0.0.1:11434")
    print(llm.invoke("Hi"))
    return llm

In [13]:
## Document loader
def document_loader(file):
    loader = PyPDFLoader(file.name)
    loaded_document = loader.load()
    return loaded_document

In [14]:
## Text splitter
def text_splitter(data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
    )
    chunks = text_splitter.split_documents(data)
    return chunks

In [15]:
## Embedding model
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()



In [16]:
## Vector db
def vector_database(chunks):
    vectordb = Chroma.from_documents(chunks, embeddings)
    return vectordb

In [17]:
## Retriever
def retriever(file):
    splits = document_loader(file)
    chunks = text_splitter(splits)
    vectordb = vector_database(chunks)
    retriever = vectordb.as_retriever()
    return retriever

In [18]:
## QA Chain
def retriever_qa(file, query):
    llm = llmmodel()
    retriever_obj = retriever(file)
    qa = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type="stuff", 
                                    retriever=retriever_obj, 
                                    return_source_documents=False)
    response = qa.invoke(query)
    return response['result']

In [23]:
# Create Gradio interface
rag_application = gr.Interface(
    fn=retriever_qa,
    # allow_flagging=.....,
    inputs=[
        gr.File(label="Upload PDF File", file_count="single", file_types=['.pdf'], type="filepath"),  # Drag and drop file upload
        gr.Textbox(label="Input Query", lines=2, placeholder="Type your question here...")
    ],
    outputs=gr.Textbox(label="Answer", lines=10),
    title="QA Bot with PDF Upload",
    description="Upload a PDF document and ask any question. The chatbot will try to answer using the provided document."
)

In [26]:
# Launch the app
rag_application.launch(server_name="127.0.0.1", server_port= 7863)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Hello, how can I help you today?
